In [5]:
from ctypes import *
import time
from sys import path
from os import sep

dwf = cdll.dwf
constants_path = "C:" + sep + "Program Files (x86)" + sep + "Digilent" + sep + "WaveFormsSDK" + sep + "samples" + sep + "py"
path.append(constants_path)
import dwfconstants as constants

In [14]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(5.0))
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

1
Power: +4.99V


In [7]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B

def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

1

In [15]:
write_register(ENABLE_REG, 0x01)
time.sleep(0.01)
write_register(ENABLE_REG, 0x03)
write_register(ATIME_REG, 0xF6)
write_register(CONTROL_REG, 0x00)

def read_colors():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    red_low = read_register(RDATAL_REG)
    red_high = read_register(RDATAH_REG)
    green_low = read_register(GDATAL_REG)
    green_high = read_register(GDATAH_REG)
    blue_low = read_register(BDATAL_REG)
    blue_high = read_register(BDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high, red_low, red_high, green_low, green_high, blue_low, blue_high]):
        clear = (clear_high << 8) | clear_low
        red = (red_high << 8) | red_low
        green = (green_high << 8) | green_low
        blue = (blue_high << 8) | blue_low
        return {'clear': clear, 'red': red, 'green': green, 'blue': blue}
    return None

In [9]:
def cleanup():
    dwf.FDwfDeviceClose(hdwf)

In [83]:

write_register(CONTROL_REG,0)
for i in range(256):
    write_register(ATIME_REG,255-i)
    colors = read_colors()

    print(f"Clear: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")

Clear: 3, R: 1, G: 2, B: 1
Clear: 13, R: 3, G: 3, B: 3
Clear: 16, R: 6, G: 7, B: 5
Clear: 20, R: 7, G: 8, B: 5
Clear: 27, R: 10, G: 10, B: 7
Clear: 31, R: 10, G: 12, B: 9
Clear: 34, R: 13, G: 15, B: 10
Clear: 42, R: 15, G: 16, B: 11
Clear: 47, R: 17, G: 19, B: 13
Clear: 54, R: 19, G: 20, B: 15
Clear: 56, R: 20, G: 23, B: 16
Clear: 65, R: 23, G: 25, B: 17
Clear: 69, R: 24, G: 27, B: 18
Clear: 75, R: 26, G: 29, B: 20
Clear: 80, R: 29, G: 32, B: 21
Clear: 84, R: 30, G: 33, B: 22
Clear: 90, R: 32, G: 35, B: 24
Clear: 96, R: 33, G: 38, B: 26
Clear: 98, R: 35, G: 39, B: 27
Clear: 104, R: 37, G: 41, B: 28
Clear: 111, R: 39, G: 43, B: 29
Clear: 116, R: 41, G: 46, B: 31
Clear: 124, R: 43, G: 47, B: 32
Clear: 126, R: 45, G: 49, B: 33
Clear: 132, R: 47, G: 51, B: 35
Clear: 137, R: 49, G: 54, B: 37
Clear: 141, R: 50, G: 55, B: 39
Clear: 142, R: 53, G: 57, B: 39
Clear: 151, R: 54, G: 60, B: 41
Clear: 155, R: 57, G: 62, B: 42
Clear: 160, R: 57, G: 62, B: 43
Clear: 166, R: 60, G: 66, B: 45
Clear: 175

Device closed
